In [8]:
# open text file
def ReadFile(filename: str):
  with open(filename, 'r') as f:
    lines = f.readlines()
  return [list(line.strip()) for line in lines]

class Grid(object):
  
  def __init__(self, lines: list):
    self.lines = lines
    self.width = len(lines[0])
    self.height = len(lines)
    self.starting_point = self.FindStartingPoint()

  def FindStartingPoint(self):
    for y, line in enumerate(self.lines):
      for x, char in enumerate(line):
        if char == 'S':
          return (x, y)
    assert False, "No starting point found"
    
  def GetFirstDirection(self):
    # Decide on first step direction
    direction = None
    # First check left side
    if self.starting_point[0] > 0:
      if self.lines[self.starting_point[1]][self.starting_point[0] - 1] in '-FL':
        return 'left'
    # Then check right side
    if self.starting_point[0] < self.width - 1:
      if self.lines[self.starting_point[1]][self.starting_point[0] + 1] in '-J7':
        return 'right'
    # Then check top side
    if self.starting_point[1] > 0:
      if self.lines[self.starting_point[1] - 1][self.starting_point[0]] in '|F7':
        return 'up'
    # Then check bottom side
    if self.starting_point[1] < self.height - 1:
      if self.lines[self.starting_point[1] + 1][self.starting_point[0]] in '|JL':
        return 'down'
      
  def GetNextPosition(self, current_position: tuple, direction: str):
    if direction == 'left':
      return (current_position[0] - 1, current_position[1])
    elif direction == 'right':
      return (current_position[0] + 1, current_position[1])
    elif direction == 'up':
      return (current_position[0], current_position[1] - 1)
    elif direction == 'down':
      return (current_position[0], current_position[1] + 1)
    else:
      assert False, "Unknown direction"
      
  def GetNextDirection(self, current_position: tuple, past_direction: str):
    current_value = self.lines[current_position[1]][current_position[0]]
    if current_value == 'F' and past_direction == 'left':
      return 'down'
    elif current_value == 'F' and past_direction == 'up':
      return 'right'
    elif current_value == 'L' and past_direction == 'left':
      return 'up'
    elif current_value == 'L' and past_direction == 'down':
      return 'right'
    elif current_value == '7' and past_direction == 'right':
      return 'down'
    elif current_value == '7' and past_direction == 'up':
      return 'left'
    elif current_value == 'J' and past_direction == 'right':
      return 'up'
    elif current_value == 'J' and past_direction == 'down':
      return 'left'
    elif current_value == '-' and past_direction == 'left':
      return 'left'
    elif current_value == '-' and past_direction == 'right':
      return 'right'
    elif current_value == '|' and past_direction == 'up':
      return 'up'
    elif current_value == '|' and past_direction == 'down':
      return 'down'
    assert False, "Incompatible direction {0} for value {1}".format(past_direction, current_value)
  
  def GetLoopLength(self):
    current_position = self.starting_point
    # Decide on first step direction
    direction = self.GetFirstDirection()
    step_count = 0
    while True:
      # Move in direction
      current_position = self.GetNextPosition(current_position, direction)
      step_count += 1
      if self.lines[current_position[1]][current_position[0]] == 'S':
        return step_count
      # Decide on next direction
      direction = self.GetNextDirection(current_position, direction)
  
  def GetZoomedInGrid(self):
    new_lines = [['.'] * (self.width * 2 + 1) for i in range(self.height * 2 + 1)]
    def MarkNewLines(new_lines: list, current_old_position: tuple, direction: str):
      old_x = current_old_position[0]
      old_y = current_old_position[1]
      new_x = current_old_position[0] * 2 + 1
      new_y = current_old_position[1] * 2 + 1
      # new_lines[current_old_position[1] * 2 + 1][current_old_position[0] * 2 + 1] = self.lines[current_old_position[1]][current_old_position[0]]
      new_lines[new_y][new_x] = self.lines[old_y][old_x]
      if direction == 'left':
        new_lines[new_y][new_x - 1] = '-'
      elif direction == 'right':
        new_lines[new_y][new_x + 1] = '-'
      elif direction == 'up':
        new_lines[new_y - 1][new_x] = '|'
      elif direction == 'down':
        new_lines[new_y + 1][new_x] = '|'
    
    # Loop through the path and fill in the new grid
    current_position = self.starting_point
    direction = self.GetFirstDirection()
    MarkNewLines(new_lines, current_position, direction)
    while True:
      # Move in direction
      current_position = self.GetNextPosition(current_position, direction)
      if self.lines[current_position[1]][current_position[0]] == 'S':
        break
      # Decide on next direction
      direction = self.GetNextDirection(current_position, direction)
      MarkNewLines(new_lines, current_position, direction)
    # join new lines
    return Grid(new_lines)
  
  def MarkOutsideCells(self):
    # Mark outside cells using breadth first search
    starting_point = (0, 0)
    queue = [starting_point]
    visited = set()
    while queue:
      current_point = queue.pop(0)
      if current_point in visited:
        continue
      visited.add(current_point)
      self.lines[current_point[1]][current_point[0]] = 'O'
      # Check left
      if current_point[0] > 0:
        left_point = (current_point[0] - 1, current_point[1])
        if left_point not in visited and self.lines[left_point[1]][left_point[0]] == '.':
          queue.append(left_point)
      # Check right
      if current_point[0] < self.width - 1:
        right_point = (current_point[0] + 1, current_point[1])
        if right_point not in visited and self.lines[right_point[1]][right_point[0]] == '.':
          queue.append(right_point)
      # Check up
      if current_point[1] > 0:
        up_point = (current_point[0], current_point[1] - 1)
        if up_point not in visited and self.lines[up_point[1]][up_point[0]] == '.':
          queue.append(up_point)
      # Check down
      if current_point[1] < self.height - 1:
        down_point = (current_point[0], current_point[1] + 1)
        if down_point not in visited and self.lines[down_point[1]][down_point[0]] == '.':
          queue.append(down_point)
          
  def GetZoomedOutGrid(self):
    new_lines = []
    for y in range(0, self.height):
      if y % 2 == 0:
        continue
      new_line = []
      for x in range(0, self.width):
        if x % 2 == 0:
          continue
        new_line.append(self.lines[y][x])
      new_lines.append(new_line)
    return Grid(new_lines)
  
  def CountDots(self):
    count = 0
    for line in self.lines:
      for char in line:
        if char == '.':
          count += 1
    return count
    
  
  def __str__(self):
    return '\n'.join([''.join(line) for line in self.lines])


def SolvePartOne(input_file: str):
  lines = ReadFile(input_file)
  grid = Grid(lines)
  return grid.GetLoopLength() // 2


def SolvePartTwo(input_file: str):
  lines = ReadFile(input_file)
  grid = Grid(lines)
  # print(grid)
  zoomed_in_grid = grid.GetZoomedInGrid()
  # print(zoomed_in_grid)
  zoomed_in_grid.MarkOutsideCells()
  # print(zoomed_in_grid)
  zoomed_out_grid = zoomed_in_grid.GetZoomedOutGrid()
  # print(zoomed_out_grid)
  return zoomed_out_grid.CountDots()
  

assert SolvePartOne('sample.txt') == 4
assert SolvePartOne('sample2.txt') == 8
part_one_solution = SolvePartOne('input.txt')
print("Part one solution: {0}".format(part_one_solution))
assert part_one_solution == 6806

assert SolvePartTwo('sample3.txt') == 4
assert SolvePartTwo('sample4.txt') == 8
assert SolvePartTwo('sample5.txt') == 10
part_two_solution = SolvePartTwo('input.txt')
print("Part two solution: {0}".format(part_two_solution))
assert part_two_solution == 449



Part one solution: 6806
Part two solution: 449
